<div align="middle">
  <h1><b><i>تمرین امتیازی</i></b></h1>
 </div>




In [ ]:
import pyspark
from pyspark import SparkContext 
from pyspark.sql import SparkSession,Row
from pyspark.sql.functions import *
import math
import pandas , numpy
import matplotlib
import pprint   

import pyspark.sql.functions as sql_func
from pyspark.sql.types import *
from pyspark.ml.recommendation import ALS, ALSModel
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.mllib.evaluation import RegressionMetrics, RankingMetrics
from pyspark.ml.evaluation import RegressionEvaluator

In [ ]:
spark = SparkSession.builder.appName("bonus").master("spark://spark-master:7077").config("fs.defaultFS","hdfs://namenode:9000/").getOrCreate()
spark

<div align="right" dir="auto">
 .در این قسمت دیتاست را لود می‌کنیم 
</div>

In [3]:
Games = spark.read.csv(f"/hw3/games.csv", header=True, inferSchema=True)
Ratings = spark.read.csv(f"/hw3/ratings.csv", header=True, inferSchema=True)

In [4]:
Games.registerTempTable("games")
Ratings.registerTempTable("ratings")

In [5]:
Games.printSchema()
Ratings.printSchema()

root
 |-- game_id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- release_date: string (nullable = true)
 |-- summary: string (nullable = true)
 |-- meta_score: string (nullable = true)

root
 |-- game_id: integer (nullable = true)
 |-- user_id: integer (nullable = true)
 |-- rating: integer (nullable = true)



In [6]:
spark.sql('select * from games limit 5;').toPandas()

,game_id,name,release_date,summary,meta_score
0,1,The Legend of Zelda: Ocarina of Time,23-Nov-98,"As a young boy, Link is tricked by Ganondorf, ...",99
1,2,Tony Hawk's Pro Skater 2,20-Sep-00,As most major publishers' development efforts ...,98
2,3,Grand Theft Auto IV,29-Apr-08,"""[Metacritic's 2008 PS3 Game of the Year; Also...",fresh off the boat from Europe. It's the hope...
3,4,SoulCalibur,8-Sep-99,"This is a tale of souls and swords, transcendi...",98
4,5,Super Mario Galaxy,12-Nov-07,[Metacritic's 2007 Wii Game of the Year] The u...,97


In [7]:
spark.sql('select * from ratings limit 5;').toPandas()

,game_id,user_id,rating
0,1,314,5
1,1,439,3
2,1,588,5
3,1,1169,4
4,1,1185,4


In [6]:
from pyspark.sql.types import IntegerType
Ratings = Ratings.withColumn("game_id", Ratings["game_id"].cast(IntegerType()))
Ratings = Ratings.withColumn("user_id", Ratings["user_id"].cast(IntegerType()))
Ratings = Ratings.withColumn("rating", Ratings["rating"].cast(IntegerType()))

In [7]:
(training, test) = Ratings.randomSplit([0.8, 0.2])

In [8]:
# Build the recommendation model using ALS on the training data
als = ALS(maxIter=2, regParam=0.01, 
          userCol="user_id", itemCol="game_id", ratingCol="rating",
          coldStartStrategy="drop",
          implicitPrefs=False)
model = als.fit(training)

# Evaluate the model by computing the RMSE on the test data
predictions = model.transform(test)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",
                                predictionCol="prediction")

rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))

Root-mean-square error = 3.7443935208179377


In [11]:
def recommendedArtists(user_id, limit=5):
    test =  model.recommendForAllUsers(limit).filter(col('user_id')==user_id).select("recommendations").collect()
    topGames = []
    for item in test[0][0]:
        topGames.append(item["game_id"])
    games_df = spark.createDataFrame(topGames, IntegerType())
    final= games_df.join(Games, games_df.value == Games["game_id"]).select(Games["game_id"], Games["name"])
    return final

recommendedArtists(314, 5).show(truncate=False)

+-------+--------------------------+
|game_id|name                      |
+-------+--------------------------+
|5784   |R-Type Final 2            |
|9904   |Honeycomb Beat            |
|3203   |Otogi 2: Immortal Warriors|
|5545   |4x4 Evolution             |
|9993   |Sega Smash Pack           |
+-------+--------------------------+

